In [4]:
import pandas as pd
from typing import Type
import sys
from typing import Dict

sys.path.append("..")
from utils import print_memory_stats, change_obj_to_cat, NpEncoder

file_name = "automotive.csv.zip"
column_types_path = "column_types.pkl"
ten_columns_path = "10_columns.csv"

# Initialize an empty dictionary to store unique values and memory usage for each column
column_info_dict: Dict[str, Dict] = {}

unique_values_dict: Dict[str, set] = {}

number_of_values: Dict[str, int] = {}

# Create a generator to read the CSV file in chunks
chunk_generator = pd.read_csv(file_name, chunksize=1_000_000, compression="zip")

# Initialize a dictionary to store total memory usage for each column
total_memory_usage = {}

# Iterate through chunks
for i, chunk in enumerate(chunk_generator):
    # Iterate through columns in each chunk
    for column in chunk.columns:
        # If the column is not in the dictionary, initialize it
        if column not in column_info_dict:
            column_info_dict[column] = {
                "memory_usage": 0,
                "memory_percent": 0,
                "dtype": chunk[column].dtype,
            }

        # Update memory usage for the current column
        column_info_dict[column]["memory_usage"] += chunk[column].memory_usage(
            deep=True
        ) // 1024

        if chunk[column].dtype == "object":

            if column not in unique_values_dict:
                unique_values_dict[column] = set()

            # Add unique values from the current chunk to the set
            unique_values_dict[column].update(chunk[column].unique())
            number_of_values[column] = number_of_values.get(column, 0) + chunk[column].count()

# Calculate the total memory usage
total_memory = sum(info["memory_usage"] for info in column_info_dict.values())

# Calculate memory percentage for each column
for column, info in column_info_dict.items():
    info["memory_percent"] = (info["memory_usage"] / total_memory) * 100

# Convert the dictionary to a DataFrame for better representation
column_info_df = pd.DataFrame.from_dict(column_info_dict, orient="index")

# Print the results
print(column_info_df)


num_unique_values = {column: len(values) for column, values in unique_values_dict.items()}

# Print the results
for column, count in num_unique_values.items():
    print(f"Column '{column}' has {count} unique values. Percent: {count / number_of_values[column]}")

/tmp/ipykernel_18510/2383961699.py:27: DtypeWarning: Columns (1,8,9,17,22,25,26,32,37,53,65,67,69,70,91,93,107,111,120,122) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunk_generator):
/tmp/ipykernel_18510/2383961699.py:27: DtypeWarning: Columns (1,8,9,17,18,26,32,37,53,65,67,69,79,91,93,107,111,120,122,133) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunk_generator):
/tmp/ipykernel_18510/2383961699.py:27: DtypeWarning: Columns (1,8,9,17,18,25,26,32,37,45,53,65,67,69,91,93,107,111,120,122,133) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunk_generator):
/tmp/ipykernel_18510/2383961699.py:27: DtypeWarning: Columns (1,8,9,17,18,25,26,32,37,45,53,65,67,69,91,93,107,111,113,120,122,133) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunk_generator):
/tmp/ipyker

                   memory_usage  memory_percent    dtype
vin                      628451        2.248487   object
stockNum                 263306        0.942062   object
firstSeen                328129        1.173988   object
lastSeen                 328129        1.173988   object
msrp                      44489        0.159174    int64
...                         ...             ...      ...
vf_WheelBaseType         182105        0.651539   object
vf_WheelSizeFront         44489        0.159174  float64
vf_WheelSizeRear          44489        0.159174  float64
vf_Wheels                 44489        0.159174  float64
vf_Windows                44489        0.159174  float64

[156 rows x 3 columns]
Column 'vin' has 1937312 unique values. Percent: 0.3401768037485415
Column 'stockNum' has 1115740 unique values. Percent: 0.29798216924951354
Column 'firstSeen' has 1562 unique values. Percent: 0.00027427495801152414
Column 'lastSeen' has 731 unique values. Percent: 0.00012835787087479137
Co